In [2]:
import sys

import ibis
import ibis.expr.datatypes as dt
import ibis.expr.schema as sch
from kafka import KafkaConsumer
from pyflink.table import EnvironmentSettings, TableEnvironment
import pandas as pd
import json

In [3]:
rows = []
consumer = KafkaConsumer("transaction")
for _, msg in zip(range(10), consumer):
    rows.append(msg)

In [4]:
rows[0].value

b'{"trans_date_trans_time": "2024-01-04 00:04:17.624", "cc_num": 4502540000000000000, "merchant": "fraud_Dare-Gibson", "category": "health_fitness", "amt": 8.95, "trans_num": "61b17bfb4709a5ec8b7c249841178db4"}'

In [5]:
import json

import pandas as pd

df = pd.DataFrame([json.loads(row.value) for row in rows])

df

,trans_date_trans_time,cc_num,merchant,category,amt,trans_num
0,2024-01-04 00:04:17.624,4502540000000000000,fraud_Dare-Gibson,health_fitness,8.95,61b17bfb4709a5ec8b7c249841178db4
1,2024-01-04 00:04:17.724,213157000000000,fraud_Terry Ltd,home,185.75,4af83d0a555949578fb4e53e9d035dad
2,2024-01-04 00:04:17.824,4890420000000000000,fraud_Yost-Rogahn,personal_care,121.79,49ed69100216291903688fb12f8ada70
3,2024-01-04 00:04:17.924,560881000000,fraud_Hills-Boyer,personal_care,12.42,2e27dbc4bd6571dd1e6ed3e684d5b5e9
4,2024-01-04 00:04:18.025,4247920000000,fraud_Boyer-Haley,travel,163.07,cf311ad4abb17a11e0ec12d056bf40d2
5,2024-01-04 00:04:18.125,4311370000000000000,"fraud_Nicolas, Hills and McGlynn",entertainment,34.35,8e04e547a918156f6a53ecb65c149a90
6,2024-01-04 00:04:18.226,36078100000000,"fraud_Kilback, Nitzsche and Leffler",travel,1.88,61866906ec3f4862a35f58a0b407695c
7,2024-01-04 00:04:18.326,3551220000000000,"fraud_Lowe, Dietrich and Erdman",kids_pets,42.93,249e46073cbdae18147583fcc7da885d
8,2024-01-04 00:04:18.427,3598900000000000,fraud_Jast and Sons,food_dining,93.33,c9e2c9283caa88b895716378e0b788f9
9,2024-01-04 00:04:18.528,4265780000000000,fraud_Schiller Ltd,personal_care,8.28,1fb6438a13c4cba1f9a5294666e6e938


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   trans_date_trans_time  10 non-null     object 
 1   cc_num                 10 non-null     int64  
 2   merchant               10 non-null     object 
 3   category               10 non-null     object 
 4   amt                    10 non-null     float64
 5   trans_num              10 non-null     object 
dtypes: float64(1), int64(1), object(4)
memory usage: 608.0+ bytes


In [7]:
consumer = KafkaConsumer("transaction")
for msg in zip(range(10), consumer):
    print(msg)

(0, ConsumerRecord(topic='transaction', partition=0, offset=8614, timestamp=1704326665663, timestamp_type=0, key=None, value=b'{"trans_date_trans_time": "2024-01-04 00:04:25.662", "cc_num": 343669000000000, "merchant": "fraud_Ullrich Ltd", "category": "kids_pets", "amt": 111.61, "trans_num": "ea40f8a01ad8708677dc404fce2e4c46"}', headers=[], checksum=None, serialized_key_size=-1, serialized_value_size=201, serialized_header_size=-1))
(1, ConsumerRecord(topic='transaction', partition=0, offset=8615, timestamp=1704326665763, timestamp_type=0, key=None, value=b'{"trans_date_trans_time": "2024-01-04 00:04:25.763", "cc_num": 6534630000000000, "merchant": "fraud_Zemlak, Tillman and Cremin", "category": "personal_care", "amt": 4.62, "trans_num": "92807a71548f70b062ab4a88d2fdb3f1"}', headers=[], checksum=None, serialized_key_size=-1, serialized_value_size=219, serialized_header_size=-1))
(2, ConsumerRecord(topic='transaction', partition=0, offset=8616, timestamp=1704326665864, timestamp_type=0,

In [8]:
local = True

# 1. create a TableEnvironment
env_settings = EnvironmentSettings.in_streaming_mode()
table_env = TableEnvironment.create(env_settings)
# write all the data to one file
table_env.get_config().set("parallelism.default", "1")

# The `flink` backend does not create `TableEnvironment` objects; pass
# the `TableEnvironment` object created above to `ibis.flink.connect`.
connection = ibis.flink.connect(table_env)

# Flink’s streaming connectors aren't part of the binary distribution.
# Link the Kafka connector for cluster execution by adding a JAR file.
connection._exec_sql("ADD JAR 'flink-sql-connector-kafka-3.0.2-1.18.jar'")

In [9]:
source_schema = sch.Schema(
    {
        'trans_date_trans_time': dt.timestamp(scale=3),
        'cc_num': dt.int64,
        'amt': dt.float64,
        'merchant': dt.str,
        'category': dt.str,   
        # 'first': dt.str,
        # 'last': dt.str,
        # 'gender': dt.str,
        # 'street': dt.str,
        # 'city': dt.str,
        # 'state': dt.str,
        # 'zipcode': dt.str,
        # 'latitude': dt.float64,
        # 'longitude': dt.float64,
        # 'city_pop': dt.int32,
        # 'job': dt.str,
        # 'dob': dt.str,
        'trans_num': dt.str,
        # 'unix_time': dt.int64,
        # 'merch_lat': dt.float64,
        # 'merch_long': dt.float64,
        # 'is_fraud': dt.int32,
    }   
)

source_configs = {
    "connector": "kafka",
    "topic": "transaction",
    "properties.bootstrap.servers": "localhost:9092" if local else "kafka:29092",
    "properties.group.id": "test_3",
    "scan.startup.mode": "earliest-offset",
    "format": "json",
}

t = connection.create_table(
    "transaction",
    schema=source_schema,
    tbl_properties=source_configs,
    watermark=ibis.watermark(
        time_col="trans_date_trans_time", allowed_delay=ibis.interval(seconds=15)
    ),
)


In [12]:
# connection.drop_table("transaction")

In [10]:
# 3. create sink Table
sink_schema = sch.Schema(
    {
        'trans_num': dt.str,
        'merchant': dt.str,
        'category': dt.str,
        'cc_num': dt.int64,
        'amt': dt.float64,
        'cc_num_amt_5m': dt.float64, 
        # 'merchant_amt_5m': dt.float64, 
        # 'category_amt_5m': dt.float64,       
    }
)

sink_configs = {
    "connector": "kafka",
    "topic": "fraud_features_topic",
    "properties.bootstrap.servers": "localhost:9092" if local else "kafka:29092",
    "format": "json",
}

connection.create_table(
    "fraud_features_table", schema=sink_schema, tbl_properties=sink_configs
)

DatabaseTable: fraud_features_table
  trans_num     string
  merchant      string
  category      string
  cc_num        int64
  amt           float64
  cc_num_amt_5m float64

In [26]:
# connection.drop_table("fraud_features_table")

In [11]:
def create_window_spec(group_by, order_by, interval_in_minutes):
    return ibis.window(
        group_by=group_by,
        order_by=order_by,
        range=(-ibis.interval(minutes=interval_in_minutes), 0),
    )
window_spec = create_window_spec(group_by=t.cc_num, order_by=t.trans_num, interval_in_minutes=5)

In [12]:
# 4. query from source table and perform calculations
agged = t[
    t.trans_num,
    t.merchant,
    t.category,
    t.cc_num.name("cc_num"),
    t.amt,
    t.amt.sum()
        .over(create_window_spec(group_by=t.cc_num, order_by=t.trans_date_trans_time, interval_in_minutes=1))
        .name("cc_num_amt_5m"),
    # t.amt.sum()
    #     .over(create_window_spec(group_by=t.cc_num, order_by=t.trans_date_trans_time, interval_in_minutes=5))
    #     .name("merchant_amt_5m"),
    # t.amt.sum()
    #     .over(create_window_spec(group_by=t.cc_num, order_by=t.trans_date_trans_time, interval_in_minutes=10))
    #     .name("category_amt_5m"),
   
]

In [13]:
agged

r0 := DatabaseTable: transaction
  trans_date_trans_time timestamp(3)
  cc_num                int64
  amt                   float64
  merchant              string
  category              string
  trans_num             string

Selection[r0]
  selections:
    trans_num:     r0.trans_num
    merchant:      r0.merchant
    category:      r0.category
    cc_num:        r0.cc_num
    amt:           r0.amt
    cc_num_amt_5m: WindowFunction(func=Sum(r0.amt), frame=RangeWindowFrame(table=r0, start=WindowBoundary(value=1 
m, preceding=True), end=WindowBoundary(Cast(0, to=interval('m'))), group_by=[r0.cc_num], order_by=[asc 
r0.trans_date_trans_time]))

In [14]:
# 5. emit query result to sink table
connection.insert("fraud_features_table", agged)

In [15]:
if local:
    # Use the Kafka Python client to stream records from the sink topic.
    # Otherwise, the mini cluster will shut down upon script completion.
    consumer = KafkaConsumer("fraud_features_topic")
    for _, msg in zip(range(10), consumer):
        print(msg)


ConsumerRecord(topic='fraud_features_topic', partition=0, offset=9686, timestamp=1704326690514, timestamp_type=0, key=None, value=b'{"trans_num":"f3870d607e888d38a79ef952b28e676b","merchant":"fraud_Lubowitz-Walter","category":"kids_pets","cc_num":4900630000000,"amt":39.64,"cc_num_amt_5m":39.64}', headers=[], checksum=None, serialized_key_size=-1, serialized_value_size=163, serialized_header_size=-1)
ConsumerRecord(topic='fraud_features_topic', partition=0, offset=9687, timestamp=1704326690514, timestamp_type=0, key=None, value=b'{"trans_num":"1887c447c6d598e2559e1b1f3f35d5f4","merchant":"fraud_Bednar PLC","category":"kids_pets","cc_num":2383460000000000,"amt":89.61,"cc_num_amt_5m":89.61}', headers=[], checksum=None, serialized_key_size=-1, serialized_value_size=161, serialized_header_size=-1)
ConsumerRecord(topic='fraud_features_topic', partition=0, offset=9688, timestamp=1704326690514, timestamp_type=0, key=None, value=b'{"trans_num":"415e39baad9608d29025e28ed99dda98","merchant":"frau